# Implementing and testing the equatorial to Galactocentric transformation in Astropy

https://docs.astropy.org/en/stable/coordinates/galactocentric.html#coordinates-galactocentric

In [1]:
%matplotlib inline

In [93]:
import numpy as np
import matplotlib.pyplot as plt


import astropy.stats as aps
import astropy.coordinates as coord
import astropy.units as u
from astropy.coordinates.builtin_frames.galactocentric \
    import get_matrix_vectors
import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

In [125]:
# #  Rotation matrix to convert to cartesian.
# def r_icrs(ra, dec, d):
#     x_icrs = np.array([d * np.cos(ra) * np.cos(dec)])
#     y_icrs = np.array([d * np.sin(ra) * np.cos(dec)])
#     z_icrs = np.array([d * np.sin(ra)])
#     return np.vstack((x_icrs, y_icrs, z_icrs))

def r_icrs(ra, dec, d):
    return np.array([
        [d * np.cos(ra) * np.cos(dec)],
        [d * np.sin(ra) * np.cos(dec)],
        [d * np.sin(ra)]
    ])

nstars = 1
ra = 30.
dec = 17.
d = 1.5

r = r_icrs(ra, dec, d)

assert np.shape(r)[0] == 3
assert np.shape(r)[1] == nstars

In /Users/rangus/Applications/anaconda3/lib/python3.7/site-packages/astropy/coordinates/builtin_frames/galactocentric.py:

In [126]:
# <Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
#     (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)>

In [127]:
# Rotation matrices to point the x-axis towards the Galactic center.
ra_gc, dec_gc = 266.4051, -28.936175
d_gc = 8.122

R1 = np.array([
    [np.cos(dec_gc), 0, np.sin(dec_gc)],
    [0, 1, 0],
    [-np.sin(dec_gc), 0, np.cos(dec_gc)]
])


R2 = np.array([
    [np.cos(ra_gc), np.sin(ra_gc), 0],
    [-np.sin(ra_gc), np.cos(ra_gc), 0],
    [0, 0, 1]
])

assert np.shape(R1) == (3, 3)
assert np.shape(R2) == (3, 3)

print(R1)
print(R2)

[[-0.78886207  0.          0.61457028]
 [ 0.          1.          0.        ]
 [-0.61457028  0.         -0.78886207]]
[[-0.80786514  0.58936739  0.        ]
 [-0.58936739 -0.80786514  0.        ]
 [ 0.          0.          1.        ]]


In [128]:
# Now align with the plane of the Galaxy, with roll angle eta.

eta = 58.5986320306

R3 = np.array([
    [1, 0, 0],
    [0, np.cos(eta), np.sin(eta)],
    [0, -np.sin(eta), np.cos(eta)]
])

assert np.shape(R3) == (3, 3)
R3

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        , -0.46104098,  0.88737884],
       [ 0.        , -0.88737884, -0.46104098]])

In [129]:
# R = R1 . R2 . R3

R1_dot_R2 = np.dot(R1, R2)
R = np.dot(R1_dot_R2, R3)

R2_dot_R3 = np.dot(R2, R3)
Rtest = np.dot(R1, R2_dot_R3)

for i in range(3):
    for j in range(3):
        assert np.isclose(R[i, j], Rtest[i, j], atol=1e-10)
assert np.shape(R) == (3, 3)

Subtract the distance to the Galactic center.

In [130]:
xhat = np.array([[1, 0, 0]]).T
print(np.shape(xhat))

print(np.shape(np.dot(R, r)))
rdash = np.dot(R, r) - d_gc * xhat
print(rdash)

(3, 1)
(3, 1)
[[-7.2661871]
 [ 1.2518674]
 [ 0.2592978]]


Account for Sun's height above the midplane.

In [131]:
zsun = 20.8 * 1e-3
print(d_gc)

theta = np.arcsin(zsun/d_gc)

H = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])

rgc = np.dot(H, rdash)

8.122


In [132]:
rgc

array([[-7.26549923],
       [ 1.2518674 ],
       [ 0.27790526]])

In [124]:
sun_xyz = [-8.122, 0, 0] * u.kpc
sun_vxyz = [12.9, 245.6, 7.78] * u.km/u.s

galcen_frame = coord.Galactocentric(galcen_distance=np.abs(sun_xyz[0]),
                                    galcen_v_sun=sun_vxyz,
                                    z_sun=0*u.pc)

# Calculate XYZ position from ra, dec and parallax
c = coord.SkyCoord(ra = ra*u.deg,
                   dec = dec*u.deg,
                   distance = d*u.kpc)
galcen = c.transform_to(galcen_frame)
print(galcen.data.xyz)

[-9.02881222  0.62237289 -1.01997234] kpc
